In [2]:
import torch

from transformers import AutoFeatureExtractor
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomDataset(Dataset):
    def __init__(self, csv_path, img_dir, feature_extractor=None, augment=False):
        self.df = pd.read_csv(csv_path)[["filename", "angle"]]
        self.df = self.df[self.df["angle"] < 360]
        self.img_dir = img_dir
        self.feature_extractor = feature_extractor
        self.augment = augment
        
        # Define base transforms (always applied)
        self.base_transform = transforms.Compose([
            transforms.Resize(256),  # Slightly larger than input size
            transforms.CenterCrop(224),  # Swin's default input size
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Define augmentation transforms
        self.augment_transform = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.df.iloc[idx]['filename'])
        with Image.open(img_name) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
                
            # Apply transforms
            if self.augment:
                img = self.augment_transform(img)
            else:
                img = self.base_transform(img)
                
        label = self.df.iloc[idx]['angle']
        return img, label

# Initialize feature extractor (for Swin Transformer)
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swin-base-patch4-window7-224")

train_csv = "/kaggle/input/smai-s-25-section-a-project-phase-2/labels_train.csv"
val_csv = '/kaggle/input/smai-s-25-section-a-project-phase-2/labels_val.csv'

train_img_dir = '/kaggle/input/smai-s-25-section-a-project-phase-2/images_train/images_train/'
val_img_dir = "/kaggle/input/smai-s-25-section-a-project-phase-2/images_val/images_val"

# Initialize datasets
train_dataset = CustomDataset(
    train_csv, 
    train_img_dir, 
    feature_extractor=feature_extractor,
    augment=True  # Enable augmentation for training
)

val_dataset = CustomDataset(
    val_csv, 
    val_img_dir, 
    feature_extractor=feature_extractor,
    augment=False  # No augmentation for validation
)

# DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

2025-05-06 02:48:49.424105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746499729.628460      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746499729.684357      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# import torchvision.transforms as transforms
# from torchvision.models import convnext_base, ConvNeXt_Base_Weights

# import numpy as np
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import time
# import math

# # Set device
# print(f"Using device: {device}")

# # Setup data loaders
# # Assuming train_dataset and val_dataset are already defined and have angle labels
# # Apply transforms
# train_dataset.transform = transform
# val_dataset.transform = transform

# batch_size=32
# num_workers=4
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=num_workers,
#     pin_memory=True if torch.cuda.is_available() else False
# )

# val_loader = DataLoader(
#     val_dataset,
#     batch_size=batch_size,
#     shuffle=False,
#     num_workers=num_workers,
#     pin_memory=True if torch.cuda.is_available() else False
# )

In [9]:
from transformers import SwinForImageClassification
import torch.nn as nn
import torch.optim as optim
import time
from tqdm.notebook import tqdm
import numpy as np

def build_angle_model(output_mode='direct'):
    """
    Build a Swin Transformer model for angle prediction.
    
    Args:
        output_mode (str): 'direct' for direct angle prediction or 
                         'sin_cos' for predicting sin and cos of the angle
    """
    # Load pre-trained Swin Transformer model
    model = SwinForImageClassification.from_pretrained("microsoft/swin-base-patch4-window7-224")
    
    # Modify the classifier head for regression
    if output_mode == 'direct':
        # Direct angle prediction (single value output)
        model.classifier = nn.Sequential(
            nn.LayerNorm(1024),
            nn.Dropout(0.2),
            nn.Linear(1024, 1)
        )
    elif output_mode == 'sin_cos':
        # Predict sin and cos of the angle
        model.classifier = nn.Sequential(
            nn.LayerNorm(1024),
            nn.Dropout(0.2),
            nn.Linear(1024, 2)
        )
    else:
        raise ValueError("output_mode must be 'direct' or 'sin_cos'")

    # Save the original forward method
    original_forward = model.forward
    
    # Define new forward method
    def new_forward(*args, **kwargs):
        outputs = original_forward(*args, **kwargs)
        return outputs.logits
    
    # Replace the forward method
    model.forward = new_forward
    
    return model, output_mode

# Custom loss function for sin-cos angle prediction
def angle_loss_fn(pred, target, output_mode='direct'):
    if output_mode == 'direct':
        # MSE loss for direct angle prediction
        # Convert angles to radians for loss calculation
        pred_rad = torch.deg2rad(pred.squeeze())
        target_rad = torch.deg2rad(target)
        
        # Calculate the angular distance (handles the circular nature of angles)
        # This computes min(|a-b|, 2π-|a-b|) which is the shortest distance on a circle
        diff = torch.abs(pred_rad - target_rad)
        angular_diff = torch.min(diff, 2 * math.pi - diff)
        
        # Mean squared angular difference
        return torch.mean(angular_diff ** 2)
    
    elif output_mode == 'sin_cos':
        # For sin-cos prediction:
        # pred is [sin, cos], target is the angle in degrees
        
        # Convert target angles to sin, cos
        target_rad = torch.deg2rad(target)
        target_sin = torch.sin(target_rad)
        target_cos = torch.cos(target_rad)
        target_sincos = torch.stack([target_sin, target_cos], dim=1)
        
        # MSE loss between predicted and target sin, cos values
        return nn.MSELoss()(pred, target_sincos)

# Function to convert sin-cos to angle
def sincos_to_angle(sin_val, cos_val):
    """Convert sin and cos values to angle in degrees"""
    angle_rad = torch.atan2(sin_val, cos_val)
    angle_deg = torch.rad2deg(angle_rad)
    # Ensure angle is in [0, 360)
    angle_deg = (angle_deg + 360) % 360
    return angle_deg

# Training function
def train_angle_model(model, train_loader, val_loader, output_mode='direct', num_epochs=10):
    model = model.to(device)
    
    # Optimizer
    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.05)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=2, verbose=True
    )
    
    # Training metrics tracking
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        start_time = time.time()
        
        # Training phase
        model.train()
        running_loss = 0.0
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        train_bar = tqdm(train_loader, desc="Training")
        
        for inputs, angles in train_bar:
            inputs, angles = inputs.to(device), angles.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = angle_loss_fn(outputs, angles, output_mode)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Track metrics
            running_loss += loss.item() * inputs.size(0)
            
            # Update progress bar
            train_bar.set_postfix(loss=loss.item())
        
        # Calculate epoch statistics
        epoch_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        angle_errors = []
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc="Validation")
            for inputs, angles in val_bar:
                inputs, angles = inputs.to(device), angles.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = angle_loss_fn(outputs, angles, output_mode)
                
                # Track metrics
                val_loss += loss.item() * inputs.size(0)
                
                # Calculate angle error for each sample
                if output_mode == 'direct':
                    predicted_angles = outputs.squeeze()
                else:  # sin_cos mode
                    sin_vals, cos_vals = outputs[:, 0], outputs[:, 1]
                    predicted_angles = sincos_to_angle(sin_vals, cos_vals)
                
                # Calculate angular difference (circular metric)
                for pred_angle, true_angle in zip(predicted_angles.cpu(), angles.cpu()):
                    # Convert to numpy for easier calculation
                    pred = pred_angle.item() % 360
                    true = true_angle.item() % 360
                    
                    # Calculate minimum angular distance
                    diff = abs(pred - true)
                    error = min(diff, 360 - diff)
                    angle_errors.append(error)
                
                # Update progress bar with current loss
                val_bar.set_postfix(loss=loss.item())
        
        # Calculate epoch statistics
        epoch_val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        mean_angle_error = np.mean(angle_errors)
        median_angle_error = np.median(angle_errors)
        
        # Update learning rate based on validation loss
        scheduler.step(epoch_val_loss)
        
        # Save best model
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), "best_convnext_angle_predictor.pth")
            print(f"New best model saved with validation loss: {best_val_loss:.4f}")
            print(f"Mean angle error: {mean_angle_error:.2f}°, Median angle error: {median_angle_error:.2f}°")
        
        # Print epoch summary
        time_taken = time.time() - start_time
        print(f"Epoch {epoch+1}/{num_epochs} completed in {time_taken:.2f}s")
        print(f"Train Loss: {epoch_train_loss:.4f}")
        print(f"Val Loss: {epoch_val_loss:.4f}")
        print(f"Mean angle error: {mean_angle_error:.2f}°, Median angle error: {median_angle_error:.2f}°")
        print("-" * 50)
    
    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig('angle_training_history.png')
    plt.show()
    
    return model, train_losses, val_losses

# Function to evaluate model on validation set
def evaluate_angle_model(model, val_loader, output_mode='direct'):
    model = model.to(device)
    model.eval()
    
    all_preds = []
    all_targets = []
    angle_errors = []
    
    with torch.no_grad():
        for inputs, angles in tqdm(val_loader, desc="Evaluating"):
            inputs, angles = inputs.to(device), angles.to(device)
            outputs = model(inputs)
            
            # Convert outputs to angles based on output mode
            if output_mode == 'direct':
                predicted_angles = outputs.squeeze()
            else:  # sin_cos mode
                sin_vals, cos_vals = outputs[:, 0], outputs[:, 1]
                predicted_angles = sincos_to_angle(sin_vals, cos_vals)
            
            # Store predictions and targets
            all_preds.extend(predicted_angles.cpu().numpy())
            all_targets.extend(angles.cpu().numpy())
            
            # Calculate angular difference for each prediction
            for pred, target in zip(predicted_angles.cpu().numpy(), angles.cpu().numpy()):
                # Ensure angles are in [0, 360)
                pred = pred % 360
                target = target % 360
                
                # Calculate minimum angular distance
                diff = abs(pred - target)
                error = min(diff, 360 - diff)
                angle_errors.append(error)
    
    # Calculate metrics
    mean_error = np.mean(angle_errors)
    median_error = np.median(angle_errors)
    percentile_90 = np.percentile(angle_errors, 90)
    
    print(f"Evaluation Results:")
    print(f"Mean Angle Error: {mean_error:.2f}°")
    print(f"Median Angle Error: {median_error:.2f}°")
    print(f"90th Percentile Error: {percentile_90:.2f}°")
    
    # Plot histogram of errors
    plt.figure(figsize=(10, 6))
    plt.hist(angle_errors, bins=36, alpha=0.7)
    plt.axvline(mean_error, color='r', linestyle='dashed', linewidth=1, label=f'Mean Error: {mean_error:.2f}°')
    plt.axvline(median_error, color='g', linestyle='dashed', linewidth=1, label=f'Median Error: {median_error:.2f}°')
    plt.axvline(percentile_90, color='b', linestyle='dashed', linewidth=1, label=f'90th Percentile: {percentile_90:.2f}°')
    plt.xlabel('Angle Error (degrees)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Angle Prediction Errors')
    plt.legend()
    plt.savefig('angle_error_distribution.png')
    plt.show()
    
    # Visualize predictions vs targets
    plt.figure(figsize=(10, 10))
    plt.scatter(all_targets, all_preds, alpha=0.3, s=10)
    plt.plot([0, 360], [0, 360], 'r--')  # Perfect prediction line
    plt.xlabel('True Angle (degrees)')
    plt.ylabel('Predicted Angle (degrees)')
    plt.title('True vs Predicted Angles')
    plt.xlim(0, 360)
    plt.ylim(0, 360)
    plt.grid(True)
    plt.savefig('angle_predictions.png')
    plt.show()
    
    return mean_error, median_error, percentile_90


# Configure parameters
batch_size = 32
num_epochs = 50
# Choose between 'direct' and 'sin_cos' prediction mode
output_mode = 'sin_cos'  # 'sin_cos' approach typically works better for angles

# Assume we have train_dataset and val_dataset already defined
# with angles as labels instead of region IDs
# Placeholder: You need to replace this with your actual dataset initialization
# train_dataset = YourDataset(...)
# val_dataset = YourDataset(...)

# Build model
model, mode = build_angle_model(output_mode=output_mode)

# Train model
model, train_losses, val_losses = train_angle_model(
    model, train_loader, val_loader, output_mode=mode, num_epochs=num_epochs
)

# Load best model
model.load_state_dict(torch.load("best_convnext_angle_predictor.pth"))

# Evaluate on validation set
mean_error, median_error, percentile_90 = evaluate_angle_model(model, val_loader, output_mode=mode)

# Save the final model with metadata
torch.save({
    'model_state_dict': model.state_dict(),
    'output_mode': mode,
    'mean_error': mean_error,
    'median_error': median_error
}, "final_convnext_angle_predictor.pth")

print(f"Model saved. Final metrics:")
print(f"Mean angle error: {mean_error:.2f}°")
print(f"Median angle error: {median_error:.2f}°")

Epoch 1/50


Training:   0%|          | 0/205 [00:00<?, ?it/s]

Validation:   0%|          | 0/12 [00:00<?, ?it/s]

New best model saved with validation loss: 0.5433
Mean angle error: 81.93°, Median angle error: 75.60°
Epoch 1/50 completed in 134.68s
Train Loss: 0.6060
Val Loss: 0.5433
Mean angle error: 81.93°, Median angle error: 75.60°
--------------------------------------------------
Epoch 2/50


Training:   0%|          | 0/205 [00:00<?, ?it/s]

...

Training:   0%|          | 0/205 [00:00<?, ?it/s]

Validation:   0%|          | 0/12 [00:00<?, ?it/s]

New best model saved with validation loss: 0.2119
Mean angle error: 32.16°, Median angle error: 19.37°
Epoch 34/50 completed in 135.56s
Train Loss: 0.0230
Val Loss: 0.2119
Mean angle error: 32.16°, Median angle error: 19.37°
--------------------------------------------------
Epoch 35/50


Training:   0%|          | 0/205 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [50]:
import os
from PIL import Image
import torch
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # adjust size as needed
    transforms.ToTensor()
])

image_dir = '/kaggle/input/test-dataset/images_test'
# image_dir = '/kaggle/input/smai-s-25-section-a-project-phase-2/images_val/images_val'
image_tensors = []

for filename in sorted(os.listdir(image_dir)):
    if filename.lower().endswith(('.jpg', '.jpeg')):
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img)
        image_tensors.append(img_tensor)
batch_tensor = torch.stack(image_tensors)  # shape: (N, C, H, W)
images = batch_tensor

# Set model to evaluation mode and disable gradients
model.eval()

# Ensure images tensor is on the same device as the model
images = images.to(device)

with torch.no_grad():
    outputs = model(images)
    
    # Convert outputs to angles based on your output mode
    if output_mode == 'direct':
        predicted_angles1 = outputs.squeeze()  # Shape: [batch_size]
    else:  # sin_cos mode
        sin_vals, cos_vals = outputs[:, 0], outputs[:, 1]
        predicted_angles = sincos_to_angle(sin_vals, cos_vals)  # Your conversion function
    
    # Convert to numpy/cpu if needed (e.g., for further processing)
    predicted_angles = predicted_angles.cpu().numpy()  # Shape: [batch_size]

In [53]:
print(len(list(predicted_angles2)))
with open("/kaggle/working/output_test.csv", 'w') as f:
    for i in range(len(list(predicted_angles))):
        f.write(f"{i+369},{int(list(predicted_angles)[i])}\n")

369
